In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
!ls '/content/drive/My Drive/MLSpring2020/TheMeanSquares-StockPrediction/Alternus-Vera TheMeanSquares/Iteration 4_Final Submission/Dataset/liar-liar processed'



'processed_LiarLiar (1).csv'   test_processed.csv    valid_processed.csv
 processed_LiarLiar.csv        train_processed.csv


In [0]:
fake_data = pd.read_csv('/content/drive/My Drive/MLSpring2020/TheMeanSquares-StockPrediction/Alternus-Vera TheMeanSquares/Iteration 4_Final Submission/Dataset/liar-liar processed/processed_LiarLiar.csv',low_memory=False)

In [0]:
fake_data.dtypes

headline_text               object
processed_headline_text     object
subject                     object
speaker                     object
speakerjobtitle             object
stateinfo                   object
partyaffiliation            object
context                     object
encoded_label                int64
tokenized                   object
lda_topics                  object
negative                   float64
neutral                    float64
positive                   float64
compound                   float64
subject_codes                int64
speaker_codes                int64
partyaffiliation_codes       int64
speakerjobtitle_codes        int64
context_codes                int64
dtype: object

In [0]:
fake_data_ = fake_data.copy()
fake_data['fake_label'] = fake_data['encoded_label'].apply(lambda x: 1 if (int(x) ==3 or int(x) == 5)  else 0)

In [0]:
fake_data.head(5)

,headline_text,processed_headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,context,encoded_label,tokenized,lda_topics,negative,neutral,positive,compound,subject_codes,speaker_codes,partyaffiliation_codes,speakerjobtitle_codes,context_codes,fake_label
0,Says the Annies List political group supports ...,nanni list polit group support third trimest a...,abortion,dwayne-bohac,State representative,Texas,republican,a mailer,5,"['nanni', 'list', 'polit', 'group', 'support',...",nanni group administr trimest third support po...,0.0,1.000,0.000,0.0000,31,868,20,930,1430,1
1,When did the decline of coal start? It started...,declin coal start start natur took start begin...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,a floor speech.,3,"['declin', 'coal', 'start', 'start', 'natur', ...",list demand third polit administr trimest supp...,0.0,0.794,0.206,0.0772,2810,2652,6,928,1257,1
2,"Hillary Clinton agrees with John McCain ""by vo...",hillari clinton agre john mccain vote give geo...,foreign-policy,barack-obama,President,Illinois,democrat,Denver,2,"['hillari', 'clinton', 'agr', 'john', 'mccain'...",trimest third polit administr support nanni li...,0.0,0.597,0.403,0.4019,3139,197,6,686,87,0
3,Health care reform legislation is likely to ma...,health care reform legisl like mandat free cha...,health-care,blog-posting,NaN,NaN,none,a news release,5,"['health', 'care', 'reform', 'legisl', 'like',...",group trimest support administr third polit na...,0.0,0.722,0.278,0.4019,3369,290,17,-1,1529,1
4,The economic turnaround started at the end of ...,econom turnaround start term,"economy,jobs",charlie-crist,NaN,Florida,democrat,an interview on CNN,3,"['econom', 'turnaround', 'start', 'term']",demand third administr trimest support polit n...,0.0,1.000,0.000,0.0000,2392,459,6,-1,3141,1


In [0]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from gensim.parsing.preprocessing import preprocess_string
from sklearn.base import BaseEstimator
from sklearn import utils as skl_utils
from tqdm import tqdm

import multiprocessing
import numpy as np

class Doc2VecTransformer(BaseEstimator):

    def __init__(self, vector_size=100, learning_rate=0.02, epochs=20,column='clean_text'):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self._model = None
        self.vector_size = vector_size
        self.workers = multiprocessing.cpu_count() - 1
        self.column = column

    def fit(self, df_x, df_y=None):
        tagged_x = [TaggedDocument(str(row[self.column]).split(), [index]) for index, row in df_x.iterrows()]
        model = Doc2Vec(documents=tagged_x, vector_size=self.vector_size, workers=self.workers)

        for epoch in range(self.epochs):
            model.train(skl_utils.shuffle([x for x in tqdm(tagged_x)]), total_examples=len(tagged_x), epochs=1)
            model.alpha -= self.learning_rate
            model.min_alpha = model.alpha

        self._model = model
        return self

    def transform(self, df_x):
        return np.asmatrix(np.array([self._model.infer_vector(str(row[self.column]).split())
                                     for index, row in df_x.iterrows()]))

In [0]:
#doc2vec_tr = Doc2VecTransformer(vector_size=300)
doc2vec_tr = Doc2VecTransformer(vector_size=10, learning_rate=0.001, epochs=10,column='processed_headline_text')
doc2vec_tr.fit(fake_data)
doc2vec_vectors = doc2vec_tr.transform(fake_data)

100%|██████████| 11507/11507 [00:00<00:00, 2823603.59it/s]


In [0]:
doc2vec_vectors.dtype

dtype('float32')

In [0]:
#doc2vec_tr = Doc2VecTransformer(vector_size=300)
doc2vector2 = Doc2VecTransformer(vector_size=10, learning_rate=0.001, epochs=10,column='lda_topics')
doc2vector2.fit(fake_data)
doc2vec_vectors2 = doc2vector2.transform(fake_data)

100%|██████████| 11507/11507 [00:00<00:00, 1949267.21it/s]


In [0]:
sentiment_values = fake_data[['compound','negative','neutral','positive']].values
sentiment_values.shape

(11507, 4)

In [0]:
cat_col_values = fake_data[['subject_codes','speaker_codes','partyaffiliation_codes','speakerjobtitle_codes','context_codes']].values
cat_col_values.shape

(11507, 5)

In [0]:
print(doc2vec_vectors.shape)
print(doc2vec_vectors2.shape)
print(sentiment_values.shape)
doc2vec_vectors3 = np.concatenate([doc2vec_vectors, doc2vec_vectors2,sentiment_values],axis=1)
print(doc2vec_vectors3.shape)

(11507, 10)
(11507, 10)
(11507, 4)
(11507, 24)


In [0]:
doc2vec_vectors4 = np.concatenate([doc2vec_vectors,sentiment_values],axis=1)
print(doc2vec_vectors4.shape)

(11507, 14)


In [0]:
print(doc2vec_vectors3.shape)

(11507, 24)


In [0]:
for v in doc2vec_vectors3:
  print(str(v))
  break

[[-0.02035626 -0.04604911 -0.0392176  -0.03392892 -0.01993528 -0.03489788
  -0.01459692  0.0374605  -0.05736085  0.02297147 -0.03491816  0.00884149
  -0.01069696 -0.01723313  0.03576699  0.01167598 -0.03011512  0.02898184
  -0.01041209 -0.0291665   0.          0.          1.          0.        ]]


In [0]:
fake_data['fake_label'] = fake_data['encoded_label'].apply(lambda x: 1 if (int(x) ==5 or int(x) == 6)  else 0)

In [0]:
labels = fake_data['fake_label']


In [0]:
labels1 = fake_data['encoded_label']


# Multi-class classification with LiarLiar dataset

In [0]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.model_selection._search import BaseSearchCV
import pickle as pkl

seed = 58

# perform the split which gets us the train data and the test data
news_train, news_test, labels_train, labels_test = train_test_split(doc2vec_vectors3, labels1,
                                                                    test_size=0.25,
                                                                    random_state=seed,
                                                                    stratify=labels)

In [0]:
import warnings
# Ignore some unimportant warnings
warnings.filterwarnings("ignore") 
                        
from mlxtend.classifier import EnsembleVoteClassifier

from sklearn.metrics import classification_report
from sklearn import model_selection
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint
from scipy.stats.distributions import uniform
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import numpy as np

In [0]:


# MLP classifier
mlp = MLPClassifier(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.8,
                    beta_2=0.9, early_stopping=False, epsilon=1e-08,
                    hidden_layer_sizes=(600, 300), learning_rate='constant',
                    learning_rate_init=0.0001, max_iter=500, momentum=0.9,
                    nesterovs_momentum=True, power_t=0.5, random_state=0, shuffle=True,
                    solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
                    warm_start=False)

# KNN classifier
knn = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
                           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
                           weights='distance')

# QDA classifier
qda = QuadraticDiscriminantAnalysis(priors=np.array([0.5, 0.5]),
                                    reg_param=0.6531083254653984, store_covariance=False,
                                     tol=0.0001)

# GDB classifier
gdb = GradientBoostingClassifier(criterion='friedman_mse', init=None,
                                 learning_rate=0.1, loss='exponential', max_depth=10,
                                 max_features='log2', max_leaf_nodes=None,
                                 min_impurity_decrease=0.0, min_impurity_split=None,
                                 min_samples_leaf=0.0012436966435001434,
                                 min_samples_split=100, min_weight_fraction_leaf=0.0,
                                 n_estimators=200, presort='auto', random_state=0,
                                 subsample=0.8, verbose=0, warm_start=False)

# SVC classifier
svc = SVC(C=0.8, cache_size=200, class_weight=None, coef0=0.0,
          decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
          max_iter=-1, probability=True, random_state=0, shrinking=True,
          tol=0.001, verbose=False)

# GNB classifier
gnb = GaussianNB(priors=None)

# RF classifier
rf = RandomForestClassifier(bootstrap=False, class_weight=None,
                            criterion='entropy', max_depth=10, max_features=7,
                            max_leaf_nodes=None, min_impurity_decrease=0.0,
                            min_impurity_split=None, min_samples_leaf=9,
                            min_samples_split=6, min_weight_fraction_leaf=0.0,
                            n_estimators=50, n_jobs=1, oob_score=False, random_state=None,
                            verbose=0, warm_start=False)

# LG classifier
lg = LogisticRegression(C=7.374558791, class_weight=None, dual=False,
                        fit_intercept=True, intercept_scaling=1, max_iter=100,
                        multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
                        solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

#classifiers_list = [mlp, knn, qda, gdb, svc, gnb, rf, lg]
#model_names = ['mlp','knn', 'qda', 'gdb', 'svc', 'gnb', 'rf', 'lg']
classifiers_list = [ svc, gnb, rf,mlp, knn, lg]
model_names = [  'svc', 'gnb', 'rf','mlp','knn', 'lg']

In [0]:
from sklearn.metrics import classification_report

# print details of testing results
i = 0
for model in classifiers_list:
    model.fit(news_train, labels_train)
    labels_pred = model.predict(news_test)
    
    # Report the metrics
    target_names = ['true', 'mostly-true','half-true','barely-true','false','pants-on-fire']
    print(model.__class__.__name__)
    print(classification_report(y_true=labels_test, y_pred=labels_pred, target_names=target_names, digits=3))
    filename = model_names[i]+'.pkl'
    pkl.dump(model, open(filename, 'wb'))
    i+=1
    

SVC
               precision    recall  f1-score   support

         true      0.000     0.000     0.000       460
  mostly-true      0.000     0.000     0.000       546
    half-true      0.212     1.000     0.349       609
  barely-true      0.000     0.000     0.000       468
        false      0.000     0.000     0.000       565
pants-on-fire      0.000     0.000     0.000       229

     accuracy                          0.212      2877
    macro avg      0.035     0.167     0.058      2877
 weighted avg      0.045     0.212     0.074      2877

GaussianNB
               precision    recall  f1-score   support

         true      0.129     0.072     0.092       460
  mostly-true      0.179     0.220     0.198       546
    half-true      0.223     0.245     0.233       609
  barely-true      0.190     0.083     0.116       468
        false      0.196     0.372     0.256       565
pants-on-fire      0.000     0.000     0.000       229

     accuracy                          0.192 

# Multi-class classification with Distillation

In [0]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.model_selection._search import BaseSearchCV
import pickle as pkl

seed = 58

# perform the split which gets us the train data and the test data
news_train, news_test, labels_train, labels_test = train_test_split(doc2vec_vectors, labels1,
                                                                    test_size=0.25,
                                                                    random_state=seed,
                                                                    stratify=labels)

In [0]:
from sklearn.metrics import classification_report

# print details of testing results
i = 0
for model in classifiers_list:
    model.fit(news_train, labels_train)
    labels_pred = model.predict(news_test)
    
    # Report the metrics
    target_names = ['true', 'mostly-true','half-true','barely-true','false','pants-on-fire']
    print(model.__class__.__name__)
    print(classification_report(y_true=labels_test, y_pred=labels_pred, target_names=target_names, digits=3))
    filename = model_names[i]+'.pkl'
    pkl.dump(model, open(filename, 'wb'))
    i+=1
    

SVC
               precision    recall  f1-score   support

         true      0.000     0.000     0.000       460
  mostly-true      0.000     0.000     0.000       546
    half-true      0.212     1.000     0.349       609
  barely-true      0.000     0.000     0.000       468
        false      0.000     0.000     0.000       565
pants-on-fire      0.000     0.000     0.000       229

     accuracy                          0.212      2877
    macro avg      0.035     0.167     0.058      2877
 weighted avg      0.045     0.212     0.074      2877

GaussianNB
               precision    recall  f1-score   support

         true      0.156     0.070     0.096       460
  mostly-true      0.181     0.196     0.188       546
    half-true      0.205     0.455     0.282       609
  barely-true      0.171     0.043     0.068       468
        false      0.225     0.244     0.234       565
pants-on-fire      0.000     0.000     0.000       229

     accuracy                          0.200 

# Multi-class classification with sentiment analysis

In [0]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.model_selection._search import BaseSearchCV
import pickle as pkl

seed = 58

# perform the split which gets us the train data and the test data
news_train, news_test, labels_train, labels_test = train_test_split(doc2vec_vectors4, labels1,
                                                                    test_size=0.25,
                                                                    random_state=seed,
                                                                    stratify=labels)

In [0]:
from sklearn.metrics import classification_report

# print details of testing results
i = 0
for model in classifiers_list:
    model.fit(news_train, labels_train)
    labels_pred = model.predict(news_test)
    
    # Report the metrics
    target_names = ['true', 'mostly-true','half-true','barely-true','false','pants-on-fire']
    print(model.__class__.__name__)
    print(classification_report(y_true=labels_test, y_pred=labels_pred, target_names=target_names, digits=3))
    filename = model_names[i]+'.pkl'
    pkl.dump(model, open(filename, 'wb'))
    i+=1
    

SVC
               precision    recall  f1-score   support

         true      0.000     0.000     0.000       460
  mostly-true      0.000     0.000     0.000       546
    half-true      0.212     1.000     0.349       609
  barely-true      0.000     0.000     0.000       468
        false      0.000     0.000     0.000       565
pants-on-fire      0.000     0.000     0.000       229

     accuracy                          0.212      2877
    macro avg      0.035     0.167     0.058      2877
 weighted avg      0.045     0.212     0.074      2877

GaussianNB
               precision    recall  f1-score   support

         true      0.144     0.080     0.103       460
  mostly-true      0.182     0.284     0.222       546
    half-true      0.233     0.223     0.228       609
  barely-true      0.207     0.064     0.098       468
        false      0.204     0.375     0.264       565
pants-on-fire      0.000     0.000     0.000       229

     accuracy                          0.198 

## Conclusion


The performance of the model is verified with varies classification model and the results are listed below

Algorithm | Accuracy
--- | ---
MLP classifier | 18%
KNN  | 16%
Gradient Boosting Classifier  | 19%
SVC  | 21%
Random Forest Classifier  | 18%
Logistic Regression  | 19%

Multi-label classification with Liar Liar dataset is not having good performance compare to Binary classifier